# Assignment_6

## Atari Breakout: Value Estimation and Policy Optimization

This assignment explores reinforcement learning methods in a simplified version of the classic Atari Breakout game. The task simulates a discrete MDP environment with limited states and actions to demonstrate and compare value estimation algorithms and control strategies. The goal is to evaluate how different approaches behave in estimating value functions and discovering optimal policies.

###Assignment Steps

-Define a Simple MDP inspired by Breakout (3 states, 3 actions).

-Implement Monte Carlo (First-visit & Every-visit) evaluation.

-Implement Temporal Difference learning (TD(0)).

-Apply Value Iteration and Policy Iteration.

-Perform Monte Carlo Control with ε-greedy strategy.

-Print and compare value functions and policies.

### Imports

In [1]:
import numpy as np
import random
from collections import defaultdict

### Defineing Simplified MDP

In [2]:
states = ['s0', 's1', 's2']  # s2 = terminal
actions = ['A1', 'A2', 'A3']

# Transition model: {state: {action: (next_state, reward)}}
P = {
    's0': {'A1': ('s1', 1), 'A2': ('s1', 5), 'A3': ('s2', 0)},
    's1': {'A1': ('s2', 0), 'A2': ('s0', 1), 'A3': ('s2', 5)},
    's2': {}
}


### Generate Episodes for Monte Carlo

In [3]:
def generate_episode(policy, start_state='s0'):
    episode = []
    state = start_state
    while state != 's2':
        action = policy[state]
        next_state, reward = P[state][action]
        episode.append((state, action, reward))
        state = next_state
    return episode

# Random policy for initial exploration
random_policy = {
    's0': random.choice(actions),
    's1': random.choice(actions)
}


### Monte Carlo First-Visit and Every-Visit

In [4]:
def monte_carlo(policy, first_visit=True, episodes=1000):
    returns = defaultdict(list)
    V = defaultdict(float)
    for _ in range(episodes):
        episode = generate_episode(policy)
        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            s, _, r = episode[t]
            G = r + G
            if first_visit and s in visited:
                continue
            visited.add(s)
            returns[s].append(G)
            V[s] = np.mean(returns[s])
    return dict(V)

V_mc_first = monte_carlo(random_policy, first_visit=True)
V_mc_every = monte_carlo(random_policy, first_visit=False)
print("V (MC first-visit):", V_mc_first)
print("V (MC every-visit):", V_mc_every)


V (MC first-visit): {'s1': np.float64(5.0), 's0': np.float64(6.0)}
V (MC every-visit): {'s1': np.float64(5.0), 's0': np.float64(6.0)}


### TD(0) Learning

In [5]:
def td_0(policy, alpha=0.1, gamma=0.9, episodes=1000):
    V = defaultdict(float)
    for _ in range(episodes):
        state = 's0'
        while state != 's2':
            action = policy[state]
            next_state, reward = P[state][action]
            V[state] += alpha * (reward + gamma * V[next_state] - V[state])
            state = next_state
    return dict(V)

V_td = td_0(random_policy)
print("V (TD(0)):", V_td)


V (TD(0)): {'s0': 5.499999999999993, 's1': 4.9999999999999964, 's2': 0.0}


### Value Iteration

In [6]:
def value_iteration(gamma=0.9, theta=1e-3):
    V = {s: 0 for s in states}
    policy = {}
    while True:
        delta = 0
        for s in ['s0', 's1']:
            best_value = float('-inf')
            best_action = None
            for a in actions:
                if a in P[s]:
                    ns, r = P[s][a]
                    v = r + gamma * V[ns]
                    if v > best_value:
                        best_value = v
                        best_action = a
            delta = max(delta, abs(V[s] - best_value))
            V[s] = best_value
            policy[s] = best_action
        if delta < theta:
            break
    return V, policy

V_vi, pi_vi = value_iteration()
print("Value Iteration V:", V_vi, "policy:", pi_vi)


Value Iteration V: {'s0': 31.0488971655632, 's1': 28.944007449006882, 's2': 0} policy: {'s0': 'A2', 's1': 'A2'}


### Policy Iteration

In [7]:
def policy_iteration(gamma=0.9, theta=1e-3):
    policy = {s: random.choice(actions) for s in ['s0', 's1']}
    V = {s: 0 for s in states}
    is_stable = False

    while not is_stable:
        # Policy Evaluation
        while True:
            delta = 0
            for s in ['s0', 's1']:
                ns, r = P[s][policy[s]]
                v = r + gamma * V[ns]
                delta = max(delta, abs(V[s] - v))
                V[s] = v
            if delta < theta:
                break

        # Policy Improvement
        is_stable = True
        for s in ['s0', 's1']:
            old_action = policy[s]
            best_action = max(actions, key=lambda a: P[s][a][1] + gamma * V[P[s][a][0]])
            policy[s] = best_action
            if old_action != best_action:
                is_stable = False

    return V, policy

V_pi, pi_pi = policy_iteration()
print("Policy Iteration V:", V_pi, "policy:", pi_pi)


Policy Iteration V: {'s0': 31.048817531607753, 's1': 28.943935778446978, 's2': 0} policy: {'s0': 'A2', 's1': 'A2'}


### Monte Carlo Control (ε-greedy)

In [8]:
def mc_control(epsilon=0.1, gamma=0.9, episodes=5000):
    Q = defaultdict(lambda: {a: 0 for a in actions})
    returns = defaultdict(lambda: {a: [] for a in actions})
    policy = {s: random.choice(actions) for s in ['s0', 's1']}

    for _ in range(episodes):
        # ε-greedy policy
        def select_action(s):
            if random.random() < epsilon:
                return random.choice(actions)
            return max(Q[s], key=Q[s].get)

        episode = []
        state = 's0'
        while state != 's2':
            action = select_action(state)
            next_state, reward = P[state][action]
            episode.append((state, action, reward))
            state = next_state

        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            s, a, r = episode[t]
            G = r + gamma * G
            if (s, a) not in visited:
                visited.add((s, a))
                returns[s][a].append(G)
                Q[s][a] = np.mean(returns[s][a])
                policy[s] = max(Q[s], key=Q[s].get)

    return policy, Q

policy_mc, Q_mc = mc_control()
print("MC Control found policy:", policy_mc)
print("Q(s0):", Q_mc['s0'])


MC Control found policy: {'s0': 'A2', 's1': 'A3'}
Q(s0): {'A1': np.float64(8.661818637346379), 'A2': np.float64(8.66869327500488), 'A3': np.float64(0.0)}
